In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../..")

from heritageconnector import datastore
from heritageconnector.config import field_mapping
from heritageconnector.best_spacy_pipeline import load_model
from heritageconnector.datastore import es, index
from smg_jobs.smg_loader import preprocess_text_for_ner

import entity_linker

import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

2021-03-11 16:45:50,444 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/


### NERLoader

In [2]:
source_description_field = target_description_field = "data.http://www.w3.org/2001/XMLSchema#description"
target_title_field = "graph.@rdfs:label.@value"
target_alias_field = "graph.@skos:altLabel.@value"
target_type_field = "graph.@skos:hasTopConcept.@value"

record_loader = datastore.RecordLoader("SMG", field_mapping)
ner_loader = datastore.NERLoader(
    record_loader,
    source_es_index = 'heritageconnector',
    target_es_index = 'heritageconnector',
    source_description_field = source_description_field, 
    target_title_field = target_title_field, 
    target_description_field = target_description_field, 
    target_type_field = target_type_field,
    target_alias_field = target_alias_field,
    entity_types_to_link={"PERSON", "OBJECT", "ORG"},
    text_preprocess_func=preprocess_text_for_ner
)

In [3]:
entity_list = ner_loader.get_list_of_entities_from_es("en_core_web_trf", 500, random_seed=420)
len(entity_list)

2021-03-11 16:45:52,920 - heritageconnector.datastore - INFO - Fetching docs and running NER.
spacy tried to use GPU but failed
2021-03-11 16:45:58,883 - hc_nlp.pipeline - INFO - Loading thesaurus from ../../../heritageconnector/../GITIGNORE_DATA/labels_all_unambiguous_types_people_orgs.jsonl
2021-03-11 16:46:00,264 - hc_nlp.pipeline - INFO - 17016 term thesaurus imported in 1s


0it [00:00, ?it/s]

1359

In [7]:
entity_list[0]

{'item_uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co426643',
 'item_description': 'Notice. British Railways (Southern Region). Make the most of your Holiday! Holiday Run-About Tickets, 1 May-29 October 1949. Blue text on white background. BR(SR) ref: AD5949/2500/27449. Printed by McCorquodale & Co Ltd, London. Format: double royal, 40 x 25 inches, 1016 x 635mm.',
 'item_description_with_ent': 'Notice. [[British Railways]] (Southern Region). Make the most of your Holiday! Holiday Run-About Tickets, 1 May-29 October 1949. Blue text on white background. BR(SR) ref: AD5949/2500/27449. Printed by McCorquodale & Co Ltd, London. Format: double royal, 40 x 25 inches, 1016 x 635mm.',
 'ent_label': 'ORG',
 'ent_text': 'British Railways',
 'ent_sentence': 'British Railways (Southern Region).',
 'ent_start_idx': 8,
 'ent_end_idx': 24}

In [4]:
entity_list_with_links = ner_loader.get_link_candidates(candidates_per_entity_mention=10)

2021-03-11 16:46:47,422 - heritageconnector.datastore - INFO - Getting link candidates for each of 1359 entities


  0%|          | 0/1359 [00:00<?, ?it/s]

In [9]:
entity_list_with_links

[{'item_uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co426643',
  'item_description': 'Notice. British Railways (Southern Region). Make the most of your Holiday! Holiday Run-About Tickets, 1 May-29 October 1949. Blue text on white background. BR(SR) ref: AD5949/2500/27449. Printed by McCorquodale & Co Ltd, London. Format: double royal, 40 x 25 inches, 1016 x 635mm.',
  'item_description_with_ent': 'Notice. [[British Railways]] (Southern Region). Make the most of your Holiday! Holiday Run-About Tickets, 1 May-29 October 1949. Blue text on white background. BR(SR) ref: AD5949/2500/27449. Printed by McCorquodale & Co Ltd, London. Format: double royal, 40 x 25 inches, 1016 x 635mm.',
  'ent_label': 'ORG',
  'ent_text': 'British Railways',
  'ent_sentence': 'British Railways (Southern Region).',
  'ent_start_idx': 8,
  'ent_end_idx': 24,
  'link_candidates': [{'uri': 'https://collection.sciencemuseumgroup.org.uk/people/cp3862',
    'title': 'British Railways',
    'descriptio

In [5]:
review_df = ner_loader.get_links_data_for_review()
# review_df.head()
# review_df.to_csv("./review_data.csv")

In [6]:
# review_df.to_excel("./review_data_1103.xlsx")